# 0. 프로젝트 개요
약 이미지로부터 **여러 알약의 위치(bounding boxes)와 이름(class)를 검출**하는 객체 검출(Object Detection) 모델을 만드는 전체 과정을 포함합니다.

### 포함된 단계
포함 내용:
1. GPU 설정  
2. Random Seed 고정 (재현성 확보)  
3. Train/Val 데이터 분리  
4. EDA  
5. COCO(JSON) → YOLO 변환  
6. YOLO 학습 (증강 포함)  
7. Validation 평가  
8. 한글 시각화  
9. 결과 해석

### 모델
YOLOv8s (Ultralytics)  
→ 사전학습된 COCO 기반 모델을 fine-tuning  


# 1. 환경 세팅

In [ ]:
!pip install ultralytics opencv-python matplotlib tqdm

In [ ]:
!apt-get update
!apt-get install -y fonts-nanum

### 1-1. 프로젝트 기본 설정 (Seed, Device, 폴더 구조)

In [ ]:
import os
import random
import numpy as np
import torch

# Seed 고정 (재현성)
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# ⚙ Device 설정
device = 0 if torch.cuda.is_available() else 'cpu'
print("사용 디바이스:", "GPU" if device==0 else "CPU (M1/M2)")

# 프로젝트 기본 경로 설정

BASE_DIR = "/Users/apple/Downloads/codeit/project/ai06-level1-project"
IMG_DIR = os.path.join(BASE_DIR, "train_images")
ANN_DIR = os.path.join(BASE_DIR, "train_annotations")

YOLO_DIR = os.path.join(BASE_DIR, "yolo_dataset")
os.makedirs(YOLO_DIR, exist_ok=True)

print("BASE_DIR =", BASE_DIR)

### 1-2. Mac 한글 폰트 적용

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc

# Mac 기본 한글 폰트
rc('font', family='AppleGothic')

# 한글 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 2. COCO JSON → YOLO Format으로 변환

### 2-1. JSON 로딩 및 변환 함수

In [ ]:
import json
import glob
import os
import shutil

def convert_coco_to_yolo(json_path, save_img_dir, save_lbl_dir):
    os.makedirs(save_img_dir, exist_ok=True)
    os.makedirs(save_lbl_dir, exist_ok=True)

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    img_info = data["images"][0]
    annos = data["annotations"]

    W, H = img_info["width"], img_info["height"]
    img_name = img_info["file_name"]

    # 이미지 복사
    src_img = os.path.join(IMG_DIR, img_name)
    dst_img = os.path.join(save_img_dir, img_name)
    if os.path.exists(src_img):
        shutil.copy(src_img, dst_img)

    # 라벨 생성
    label_path = os.path.join(save_lbl_dir, img_name.replace(".png", ".txt"))

    with open(label_path, "w", encoding="utf-8") as f:
        for a in annos:
            cid = a["category_id"]
            x, y, w, h = a["bbox"]

            xc = (x + w/2) / W
            yc = (y + h/2) / H
            wn = w / W
            hn = h / H

            f.write(f"{cid} {xc} {yc} {wn} {hn}\n")

### 2-2. 모든 JSON 대상 변환 수행 (Train/Val Split 포함)

In [ ]:
# 모든 JSON 불러오기
all_jsons = glob.glob(os.path.join(ANN_DIR, "**/*.json"), recursive=True)
len(all_jsons)

Train : Validation = 8:2 split

In [ ]:
random.shuffle(all_jsons)

split_idx = int(len(all_jsons) * 0.8)
train_jsons = all_jsons[:split_idx]
val_jsons   = all_jsons[split_idx:]

len(train_jsons), len(val_jsons)

변환 실행

In [ ]:
paths = {
    "train_img": os.path.join(YOLO_DIR, "images/train"),
    "val_img": os.path.join(YOLO_DIR, "images/val"),
    "train_lbl": os.path.join(YOLO_DIR, "labels/train"),
    "val_lbl": os.path.join(YOLO_DIR, "labels/val"),
}

for j in train_jsons:
    convert_coco_to_yolo(j, paths["train_img"], paths["train_lbl"])

for j in val_jsons:
    convert_coco_to_yolo(j, paths["val_img"], paths["val_lbl"])

print("변환 완료")

# 3. dataset.yaml 자동 생성

### 3-1. 자동 이름 생성 코드

In [ ]:
category_map = {}  # {category_id: class_name}

json_files = glob.glob(os.path.join(ANN_DIR, "**/*.json"), recursive=True)

for jf in json_files:
    with open(jf, "r", encoding="utf-8") as f:
        data = json.load(f)
        for cat in data.get("categories", []):
            cid = cat["id"]
            cname = cat["name"]
            category_map[cid] = cname

# 정렬 (YOLO는 index 0부터 시작해야 하므로)
sorted_categories = sorted(category_map.items(), key=lambda x: x[0])

names = [name for _, name in sorted_categories]

print("총 클래스 수:", len(names))
print(names[:10])  # 일부만 보기

### 3-3. dataset.yaml 자동 생성

In [ ]:
DATASET_YAML = os.path.join(BASE_DIR, "dataset.yaml")

TRAIN_PATH = os.path.abspath(paths["train_img"])
VAL_PATH = os.path.abspath(paths["val_img"])

dataset_yaml_content = f"train: {TRAIN_PATH}\nval: {VAL_PATH}\n\n"
dataset_yaml_content += f"nc: {len(names)}\n\nnames:\n"

for n in names:
    dataset_yaml_content += f"  - {n}\n"

with open(DATASET_YAML, "w", encoding="utf-8") as f:
    f.write(dataset_yaml_content)

print("dataset.yaml 생성 완료 →", DATASET_YAML)

# 4. YOLO 학습

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

results = model.train(
    data=DATASET_YAML,
    epochs=20,
    imgsz=640,
    batch=8,
    device=device,
    name="pill_yolo_final",

    # 약 이미지 전용 Augmentation
    hsv_h=0.005,
    hsv_s=0.3,
    hsv_v=0.2,
    degrees=5,
    translate=0.05,
    scale=0.1,
    fliplr=0.5,
    flipud=0.0,
    mosaic=0.7,
    mixup=0.05,

    seed=seed,
    deterministic=True,
)

# 5. 한글 폰트 적용 + YOLO 텍스트 제거 시각화

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc
from ultralytics import YOLO

# 한글 폰트 설정
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 학습된 모델 불러오기
model = YOLO("runs/detect/pill_yolo_final/weights/best.pt")

# 클래스 이름 불러오기
class_names = model.model.names

# 6. 테스트 이미지 추론

In [ ]:
import cv2

test_img_path = os.path.join(BASE_DIR, "test_images/sample1.png")

img = cv2.imread(test_img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img_rgb, verbose=False)
boxes = results[0].boxes

# YOLO 텍스트 제거(labels=False)
annotated = results[0].plot(labels=False)
annot_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(6,8))
plt.imshow(annot_rgb)
plt.axis("off")

for b in boxes:
    x1, y1, x2, y2 = map(int, b.xyxy[0])
    cls = int(b.cls[0])
    conf = float(b.conf[0])

    label = f"{class_names[cls]} ({conf:.2f})"

    plt.text(
        x1, y1-10,
        label,
        fontsize=14,
        color="red",
        fontweight="bold",
        backgroundcolor="white",
    )

plt.show()

# 7. 모델 저장 및 불러오기

In [ ]:
best_model_path = "runs/detect/pill_yolo_final/weights/best.pt"
model = YOLO(best_model_path)

print("모델 로드 완료:", best_model_path)

# 8. Validation Dataset 성능 측정(mAP, Precision, Recall)

In [ ]:
metrics = model.val()
metrics

# 9. Validation 이미지 예측 + 시각화

In [ ]:
# 한글 폰트
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

class_names = model.model.names  # YOLO 내부 클래스명


def visualize_prediction(model, img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # YOLO 추론
    results = model(img_rgb, verbose=False)
    boxes = results[0].boxes

    # YOLO 텍스트 제거
    annotated = results[0].plot(labels=False)
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(6,8))
    plt.imshow(annotated)
    plt.axis("off")

    # 한글 텍스트 추가
    for b in boxes:
        x1, y1, x2, y2 = map(int, b.xyxy[0])
        cls = int(b.cls[0])
        conf = float(b.conf[0])

        label = f"{class_names[cls]} ({conf:.2f})"

        plt.text(
            x1, y1-10,
            label,
            fontsize=14,
            color="red",
            backgroundcolor="white",
            fontweight="bold",
        )

    plt.show()

# 10. Validation 이미지 테스트

In [ ]:
val_img_dir = "/Users/apple/Downloads/codeit/project/ai06-level1-project/yolo_dataset/images/val"
val_imgs = os.listdir(val_img_dir)[:5]

for img in val_imgs:
    visualize_prediction(model, os.path.join(val_img_dir, img))

# 11. Test Dataset 시각화

In [ ]:
test_dir = "/Users/apple/Downloads/codeit/project/ai06-level1-project/test_images"
test_imgs = os.listdir(test_dir)[:10]

for img in test_imgs:
    visualize_prediction(model, os.path.join(test_dir, img))

# 12. 개별 이미지 예측 API (앱에서 쓸 수 있는 방식)

In [ ]:
def predict_pills(model, img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = model(img_rgb, verbose=False)
    output = []

    for b in results[0].boxes:
        x1, y1, x2, y2 = map(int, b.xyxy[0])
        cls = int(b.cls[0])
        conf = float(b.conf[0])

        output.append({
            "class": class_names[cls],
            "confidence": round(conf, 3),
            "bbox": [x1, y1, x2, y2]
        })
    return output


predict_pills(model, "/Users/apple/Downloads/codeit/project/test_images/sample.png")